In [11]:
# Imports
import os, sys
import pandas as pd
from textblob import TextBlob

# Step 1: Data Preprocessing

In [14]:
"""
    Local Dataset Importing
        Originally tried to use the Kaggle API however quickly faced
        issues with rate limiting. Local downloading will speed up
        training at the cost of repeatability for others viewing project.
        
        Solution, I pull from downloaded datasets in parent folder to this
        github repository. news_data contains only the news dataset .json file,
        stock data contains a .csv with the metadata as well as 2 subfolders
        ETF and Stocks that contain their respective ticker histories
"""
news_path = os.getcwd() + "/../news_data/"
stock_path = os.getcwd() + "/../stock_data/"

news_df = pd.read_json(f'{news_path}News_Category_Dataset_v3.json', lines=True)
stock_meta = pd.read_csv(f'{stock_path}symbols_valid_meta.csv')

In [3]:
"""
    Capture all Data Frames
        Maintain a dictionary with keys as stock ticker symbols and values
        as the DataFrames captured from reading the stock's .csv
"""
stock_tickers = [x['Symbol'] for _, x in stock_meta.iterrows() if x['ETF'] == 'N'] # Remove ETFs


stock_dfs = {}
for s in stock_tickers:
    try:
        stock_dfs[s] = pd.read_csv(f"{stock_path}/stocks/{s}.csv")
    except Exception:
        print(f"Error with stock: {s}")


Error with stock: AGM$A
Error with stock: CARR.V
Error with stock: UTX.V


In [4]:
"""
    Dataset Columns
"""
print(f"News Columns: {news_df.columns.values}")
print(f"Stock Columns: {stock_dfs['A'].columns.values}")

News Columns: ['link' 'headline' 'category' 'short_description' 'authors' 'date']
Stock Columns: ['Date' 'Open' 'High' 'Low' 'Close' 'Adj Close' 'Volume']


In [5]:
"""
    Helper Functions
"""

# Return the next trading day, avoiding holidays and weekends
#   Inputs
#       cur_day      -> The current date
#       trading_days -> List of all open dates in the range
#   Returns:
#       The next available date (datetime object)
def next_trading_day(cur_day, trading_days):
    days_left = trading_days[trading_days > cur_day]
    return days_left.min() if len(days_left) else trading_days.max()

In [6]:
"""
    Prepare Stock Datasets
        Drop irrelevant columns --> High, Low, Volume, adj_close
        Drop information prior to 2012-01-28
        Create stock return time horizon features
"""
processed_stocks = {}
tickers = stock_dfs.keys()

for s in tickers:
    df = stock_dfs[s].copy()

    df['Ticker'] = s

    df['Date'] = pd.to_datetime(df['Date'])
    df = df[df['Date'] >= pd.Timestamp("2012-01-28")].reset_index(drop=True)
    df = df.drop(['High', 'Low', 'Adj Close', 'Volume'], axis=1)

    if df.shape[0] != 2057: # 2057 dates 2012-01-28 and 2020-04-01, cut unfilled stocks
        continue

    df['r_0d'] = (df['Close'] - df['Open'])/df['Open']
    df['r_1d'] = df['Close'].shift(-1).pct_change(periods=1, fill_method=None)
    df['r_7d'] = df['Close'].shift(-7).pct_change(periods=7, fill_method=None)
    df['r_30d'] = df['Close'].shift(-30).pct_change(periods=30, fill_method=None)

    processed_stocks[s] = df

print(f"Number of Stocks before processing: \t{len(stock_dfs)}")
print(f"Number of Stocks after processing: \t{len(processed_stocks)}")

stock_dfs = {k: v for k, v in processed_stocks.items()}

Number of Stocks before processing: 	5881
Number of Stocks after processing: 	3511


In [7]:
print(stock_dfs['A'])

           Date       Open      Close Ticker      r_0d      r_1d  r_7d  r_30d
0    2012-01-30  29.992847  30.872675      A  0.029335       NaN   NaN    NaN
1    2012-01-31  30.994278  30.379112      A -0.019848  0.040499   NaN    NaN
2    2012-02-01  30.815451  31.609442      A  0.025766 -0.010636   NaN    NaN
3    2012-02-02  31.623749  31.273247      A -0.011084  0.035682   NaN    NaN
4    2012-02-03  31.759657  32.389126      A  0.019820 -0.013693   NaN    NaN
...         ...        ...        ...    ...       ...       ...   ...    ...
2052 2020-03-26  70.000000  73.720001      A  0.053143 -0.038117   NaN    NaN
2053 2020-03-27  71.550003  70.910004      A -0.008945  0.024820   NaN    NaN
2054 2020-03-30  71.059998  72.669998      A  0.022657 -0.014449   NaN    NaN
2055 2020-03-31  72.339996  71.620003      A -0.009953 -0.037699   NaN    NaN
2056 2020-04-01  69.470001  68.919998      A -0.007917       NaN   NaN    NaN

[2057 rows x 8 columns]


In [25]:
"""
    Prepare News Dataset
        News DF starts 2022-09-23, ends 2012-01-28 --> First must reverse dataset
        Drop categories to only necessary --> Category, Headline, Date
        Shift dates to align with trading days, skipping weekends and holidays till next open day
"""
news_df = news_df.sort_values(by='date', ascending=True).reset_index(drop=True)
news_df = news_df.drop(['link', 'short_description', 'authors'], axis=1)
news_df = news_df[news_df['date'] <= pd.Timestamp("2020-04-01")].reset_index(drop=True)

trading_days = pd.to_datetime(stock_dfs['A']['Date'].unique())
news_df['effective_date'] = news_df['date'].apply(lambda d: next_trading_day(d, trading_days))

# Step 2: Sentiment Analysis

In [ ]:
# Create sentiment of each headline using 
news_df['sentiment'] = news_df['headline'].apply(lambda x: TextBlob(x).sentiment.polarity)

In [38]:
daily_sentiment = (
    news_df.groupby(['effective_date', 'category'])
    .agg(
        avg_sentiment = ('sentiment', 'mean'),
        article_count = ('headline', 'count')
    )
    .reset_index()
)

In [42]:
all_dates = pd.date_range(news_df['effective_date'].min(), news_df['effective_date'].max())
all_cats = news_df['category'].unique()


all_cats_per_date = pd.MultiIndex.from_product(
    [all_dates, all_cats],
    names=['date', 'category']
)

daily_sentiment = (
    daily_sentiment
    .set_index(['effective_date', 'category'])
    .reindex(all_cats_per_date)
    .fillna({'avg_sentiment': 0, 'article_count': 0})
    .reset_index()
)

In [ ]:
print(type(daily_sentiment))

             date        category  avg_sentiment  article_count
0      2012-01-30          SPORTS       0.152104            9.0
1      2012-01-30         SCIENCE       0.233333            6.0
2      2012-01-30          COMEDY       0.014242           11.0
3      2012-01-30       PARENTING       0.200000            5.0
4      2012-01-30  STYLE & BEAUTY      -0.054545           14.0
...           ...             ...            ...            ...
125365 2020-04-01       GOOD NEWS       0.000000            0.0
125366 2020-04-01  ARTS & CULTURE       0.000000            0.0
125367 2020-04-01   THE WORLDPOST       0.000000            0.0
125368 2020-04-01      WORLD NEWS       0.000000            0.0
125369 2020-04-01       U.S. NEWS       0.000000            0.0

[125370 rows x 4 columns]
